In [1]:
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join("../lib"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pk_tools import pk_tools
from shrinkage_estimators import NERCOME

In [2]:
def get_fit_selection(krange, kmin = 0.0, kmax = 0.4, pole_selection = [True, True, True, True, True]):
    k_fit_selection = np.logical_and(kmin<=krange,krange<=kmax)
    pole_fit_selection = np.repeat(pole_selection, len(krange)/len(pole_selection))
    fit_selection = k_fit_selection * pole_fit_selection
    
    return fit_selection

In [3]:
data_path = "../data/BOSS_DR12_NGC_z1/Patchy_V6C_BOSS_DR12_NGC_z1"
all_filenames = os.listdir(data_path)

p = 30
n = 100
indices = np.random.choice(len(all_filenames), n, replace=False)
filenames = np.array(all_filenames)[indices]

# Empty power spectrum matrix
P = np.empty((p, len(filenames)))

i = 0;
for filename in filenames:
    # Load the data using Beutler's pk_tools module 
    data = pk_tools.read_power(os.path.join(data_path, filename), combine_bins=10)
    kbins, pk_data_vector = pk_tools.dict_to_vec(data)
    
    fit_selection = get_fit_selection(kbins, kmin=0.0, kmax=0.1, pole_selection=[True, False, True, False, True])
    fit_pk_data_vector = pk_data_vector[fit_selection]

    P[:,i] = fit_pk_data_vector
    i += 1

print(P.shape)

(30, 100)


In [4]:
def linear_shrinkage(P):
    P_mean = np.sum(P, axis=1)/n # Find mean of each row
    P_mean_matrix = np.tile(P_mean, (n, 1)).T # Repeat mean values as columns in a p x n matrix
    X = P - P_mean_matrix

    W = []
    # Generate W array (which is 3D) of size (n, p, p), order of indices (k, i, j)
    for k in range(n):
        w = X[:,k]
        W.append(np.outer(w, w))
    W_mean = np.sum(W, axis=0)/n

    # Emperically estimated covariance matrix
    S = n / (n-1) * W_mean
    
    # Take as Target the diagonal elements of the sample cov matrix
    T = np.diag(np.diag(S))
    
    W_mean_rep = np.tile(W_mean, (n, 1, 1))
    V = W - W_mean_rep
    # Compute variance of elements of the covariance matrix
    Var = n / (n-1)**3 * np.sum(V**2, axis=0)

    # Compute estimated shrinkage intensity parameter lambda
    lmbda_est = np.sum(Var-np.diag(np.diag(Var))) / np.sum((T-S)**2)
    
    # Restrict shrinkage intensity to interval [0,1]
    if lmbda_est < 0:
        lmbda_est = 0
    elif lmbda_est > 1:
        lmbda_est = 1

    # Compute shrinkage covariance matrix
    cov_shrinkage = lmbda_est*T + (1-lmbda_est)*S
    
    return cov_shrinkage, lmbda_est

In [5]:
C_nercome, S, s_min = NERCOME.NERCOME(P, all_splits=True)
C_shrinkage, lmbda_est = linear_shrinkage(P)

In [6]:
np.savetxt(f"../data/BOSS_DR12_NGC_z1/BOSS_DR12_NGC_z1_cov/BOSS_DR12_NGC_z1_cov_{p}_{p}_sample_{n}.matrix", S)
np.savetxt(f"../data/BOSS_DR12_NGC_z1/BOSS_DR12_NGC_z1_cov/BOSS_DR12_NGC_z1_cov_{p}_{p}_NERCOME_{n}.matrix", C_nercome, header=f"s={s_min}")
np.savetxt(f"../data/BOSS_DR12_NGC_z1/BOSS_DR12_NGC_z1_cov/BOSS_DR12_NGC_z1_cov_{p}_{p}_shrinkage_{n}.matrix", C_shrinkage, header=f"lambda={lmbda_est}")

In [7]:
print(s_min)
print(lmbda_est)

82
0.492365075525228
